# **<u>Flujo de trabajo para crear cuadrículas de 100 x 100 m</u>**

---

* La idea es crear cuadrículas de de 100 x 100 para los Espacios Naturales Protegidos de la isla de Gran Canaria usando Python, garantizando la reproducibilidad de la creación de estas cuadrículas. 

---

In [1]:
## Librerías a usar:
import os
import requests
from zipfile import ZipFile
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import fiona

In [2]:
## Se usará el paquete de fiona para poder pasar los archivos SHP a KML
fiona.supported_drivers['KML'] = 'rw'

## **Descargar datos IDECanarias**

### Capa a usar: <u>**Red de Espacios Naturales Protegidos de Canarias (ENP)**</u>

In [18]:
## Variables para la descarga de datos definiendo rutas y URL del archivo
url_enp = "https://opendata.sitcan.es/upload/medio-ambiente/eennpp.zip"
carpeta_data = "data/"
carpeta_destino = "data/enps_canarias/"
ruta_zip = os.path.join(carpeta_data, "enps_canarias/enp.zip")

## Creamos la carpeta de destino ("data") si todavía no existe
for carpeta in [carpeta_data, carpeta_destino]:
    os.makedirs(carpeta, exist_ok=True)

## Proceso para descargar el archivo
response = requests.get(url=url_enp)
with open(ruta_zip, "wb") as file:
    file.write(response.content)

## Proceso para descomprimir el archivo
with ZipFile(ruta_zip, "r") as archivo_zip:
    archivo_zip.extractall(carpeta_destino)

print(f"""
> Archivos  descargado y descomprimido en '{carpeta_destino}'
> esta es la lista de los archivos de la carpeta: 
{os.listdir("data/enps_canarias/")}
""")


> Archivos  descargado y descomprimido en 'data/enps_canarias/'
> esta es la lista de los archivos de la carpeta: 
['eennpp.dbf', 'eennpp.prj', 'eennpp.shp', 'eennpp.shx', 'enp.zip']



## **Preprocesado de la capa SHP**

* Se filtrará de la capa SHP de los ENP de Canarias para obtener específicamente un archivo SHP de los <u>**ENP de Gran Canaria**</u>

In [24]:
## Cargamos los datos de los ENP de Canarias
enps_canarias = gpd.read_file("data/enps_canarias/eennpp.shp")
enps_canarias.head() ## vemos los primeros valores

,codigo,categoria,nombre,geometry
0,H-6,Paisaje Protegido,Ventejís,"POLYGON ((207380.300 3077264.524, 207361.430 3..."
1,H-7,Paisaje Protegido,Timijiraque,"POLYGON ((212318.690 3076408.720, 212322.130 3..."
2,H-3,Reserva Natural Especial,Tibataje,"POLYGON ((205028.180 3074960.459, 205020.849 3..."
3,H-2,Reserva Natural Integral,Roques de Salmor,"MULTIPOLYGON (((205086.150 3081134.320, 205088..."
4,H-5,Monumento Natural,Las Playas,"MULTIPOLYGON (((209998.290 3070692.910, 209998..."


In [28]:
## Filtramos del campo "codigo" aquellos que empiecen por la letra "C" (Gran Canaria)
enps_gran_canaria = enps_canarias[enps_canarias["codigo"].str.startswith("C")]
## Vemos de nuevo los primeros valores para ver los valores "C" de Gran Canaria en "codigo" 
enps_gran_canaria.head()

,codigo,categoria,nombre,geometry
20,C-32,Sitio de Interés Científico,Juncalillo del Sur,"POLYGON ((453679.696 3074570.097, 453669.984 3..."
21,C-31,Sitio de Interés Científico,Roque de Gando,"POLYGON ((464665.536 3090497.440, 464658.994 3..."
22,C-30,Sitio de Interés Científico,Tufia,"POLYGON ((462277.610 3093302.880, 462281.280 3..."
23,C-29,Sitio de Interés Científico,Jinámar,"POLYGON ((460256.224 3101297.088, 460260.274 3..."
24,C-28,Paisaje Protegido,Montaña de Agüimes,"POLYGON ((458220.295 3087279.345, 458220.210 3..."


## **Procesado de la capa anterior**

In [30]:
filtrado1 = enps_gran_canaria[enps_gran_canaria["codigo"].isin([
      "C-01", "C-21", "C-20", 
      "C-14", "C-15", "C-02", 
      "C-05", "C-04"
])] 

filtrado2 = gpd.overlay(enps_gran_canaria, filtrado1, how="difference")

enps_gran_canaria_procesado = pd.concat([filtrado1, filtrado2], ignore_index=True)

### **Crear la capa de cuadrículas**

* Se hará una función para cuyo objetivo sea la creación de una capa de cuadrículas para un GeoDataFrame especificado

In [31]:
def crear_cuadriculas(gdf):
    # Crear las separaciones para los municipios de Gran Canaria
    minx, miny, maxx, maxy = gdf.total_bounds

    # Generar una lista con las cuadrículas
    boxes = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            boxes.append(box(x, y, x + 100, y + 100))
            y += 100
        x += 100

    grid = gpd.GeoDataFrame({'geometry': boxes}, crs=gdf.crs)

    return grid

Con la función anterior, crearemos los **<u>KML</u>** para cada uno de los espacios con cuadrículas de 100 x 100 metros.  

In [34]:
os.makedirs("results/", exist_ok=True)

for codigo in enps_gran_canaria_procesado["codigo"].unique():
    enp_unico = enps_gran_canaria_procesado[enps_gran_canaria_procesado["codigo"] == codigo]
    enp_grid = crear_cuadriculas(enp_unico)
    enp_grid = enp_grid[enp_grid.intersects(enp_unico.unary_union)]
    enp_grid_union = gpd.overlay(enp_unico, enp_grid, how="union", keep_geom_type=True)
    enp_grid_union.to_file(f"results/{codigo}.kml", driver='KML')
